In [1]:
include("functions.jl")

genetique (generic function with 1 method)

# 1) Heuristique gloutonne

L'algorithme glouton choisi:
 - assigne successivement une couleur à chaque nœud
 - les nœuds sont traités dans l'ordre de degré décroissant
 - en choisissant pour chaque nœud la couleur minimisant le nombre de conflits avec ses voisins.

In [2]:
# Comparaison d'une solution aléatoire et d'une solution gloutonne
begin
    instance = instance_list[3]
    aleatoire = sol_alea(instance)

    println(instance.name,"\tk=",instance.k)
    println("SOLUTION ALÉATOIRE:")
    println(aleatoire.nodecolors)
    print("collisions: ")
    println(nbr_collision(instance,aleatoire))
    gloutonne = glouton(instance)
    println("\nSOLUTION GLOUTONNE:")
    println(gloutonne.nodecolors)
    print("collisions: ")
    println(nbr_collision(instance,gloutonne))
end

dsjc125.1.col	k=5
SOLUTION ALÉATOIRE:


[5, 5, 5, 3, 4, 4, 3, 4, 1, 3, 4, 1, 4, 2, 4, 3, 2, 1, 4, 3, 4, 1, 1, 5, 2, 2, 5, 1, 5, 5, 5, 1, 5, 5, 3, 4, 1, 2, 4, 5, 4, 2, 5, 2, 5, 1, 1, 1, 2, 1, 2, 1, 3, 2, 3, 2, 4, 4, 2, 1, 2, 3, 2, 1, 5, 5, 3, 1, 5, 1, 1, 5, 2, 1, 2, 2, 1, 2, 4, 1, 5, 1, 3, 2, 5, 5, 2, 3, 3, 2, 5, 4, 3, 4, 1, 2, 4, 3, 4, 1, 1, 5, 2, 4, 5, 3, 4, 3, 1, 3, 3, 4, 4, 4, 2, 5, 2, 5, 4, 3, 4, 1, 4, 3, 1]
collisions: 

160



SOLUTION GLOUTONNE:
[3, 2, 2, 4, 2, 5, 4, 3, 4, 3, 2, 5, 3, 3, 2, 4, 4, 4, 3, 5, 5, 5, 5, 4, 5, 2, 3, 3, 2, 5, 4, 3, 4, 4, 3, 5, 4, 2, 4, 5, 5, 4, 2, 2, 5, 4, 3, 5, 3, 2, 2, 2, 4, 3, 4, 4, 4, 5, 5, 2, 2, 2, 5, 3, 3, 5, 2, 3, 2, 2, 3, 3, 3, 2, 3, 2, 3, 5, 4, 4, 4, 3, 4, 2, 2, 2, 5, 3, 3, 3, 2, 4, 3, 3, 5, 5, 4, 2, 5, 3, 2, 3, 4, 5, 2, 5, 5, 3, 4, 4, 3, 2, 2, 5, 2, 4, 5, 5, 2, 4, 4, 3, 2, 2, 3]
collisions: 47


## Tests numériques de l'heuristique gloutonne

In [3]:
begin
    nsamples = 10
    println("INSTANCE NAME AND k\tMIN CONFL\tMEAN CONFL\tMAX CONFL\tTOTAL TIME\tTIME BEST SOL\tSOLS PER SECOND")
    for instance ∈ instance_list
        conflicts_samples = Vector{Int}(undef,nsamples)
        time_samples = zeros(nsamples)
        for i = 1:nsamples
            solution = glouton(instance)
            start_time = time_ns()
            conflicts = nbr_collision(instance,solution)
            run_time = time_ns()-start_time
            conflicts_samples[i] = conflicts
            time_samples[i] = run_time*1e-9
        end
        println(
            instance.name," k=",instance.k,"\t",
            minimum(conflicts_samples),"\t",
            sum(conflicts_samples)/nsamples,"\t",
            maximum(conflicts_samples),"\t",
            sum(time_samples),"\t",
            time_samples[argmin(conflicts_samples)],"\t",
            nsamples/sum(time_samples)
        ) # solutions per second irrelevant, isn't it? (since there is no local search loop)
    end
end

INSTANCE NAME AND k	MIN CONFL	MEAN CONFL	MAX CONFL	TOTAL TIME	TIME BEST SOL	SOLS PER SECOND


flat300_26_0.col k=26	226	226.0	226	0.0028273	0.000277	3536.9433735365897
le450_15c.col k=15	362	362.0	362	0.004227399999999999	0.0005432000000000001	2365.520177887118
dsjc125.1.col k=5	47	47.0	47	0.00014480000000000002	2.05e-5	69060.77348066297
dsjc125.9.col k=44	21	21.0	21	0.00019329999999999998	1.8900000000000002e-5	51733.05742369375
dsjc250.1.col k=8	88	88.0	88	0.0006982000000000001	0.0001033	14322.543683758235


dsjc250.9.col k=72	61	61.0	61	0.0008389000000000001	7.22e-5	11920.371915603766
dsjc250.5.col k=28	85	85.0	85	0.0023713000000000007	0.0001619	4217.096107620291


dsjc1000.5.col k=86	378	378.0	378	0.0327338	0.0036867000000000002	305.4946263495225


dsjc1000.5.col k=85	398	398.0	398	0.0418594	0.0041046	238.89496743861594


dsjc1000.5.col k=84	412	412.0	412	0.034967000000000005	0.0031748	285.983927703263


# 2) Structure de voisinage

On choisit d'utiliser un opérateur de voisinage simple:
 - Appliquer la couleur $i$ au nœud $j$ choisis de manière à minimiser le nombre de conflits.
   - On choisit un couple $(i,j)$ au hasard parmi tous les couples minimisant le nombre de conflits.
   - On tente également d'éviter que la nouvelle couleur soit la même que l'ancienne si possible.
   - Pour cela, on maintient une matrice des conflits stockant pour chaque nœud $j$ et chaque couleur $i$ le nombre de conflits en ce nœud $j$ si l'on lui applique la couleur $i$.

On obtient donc une structure de voisinage qui contient toutes les solutions dont un unique nœud diffère par sa couleur de la solution d'origine.

In [4]:
# Test de descente locale simple
begin
    instance = instance_list[3]
    solution = sol_alea(instance)

    for t = 1:25
        simple_neighbor(instance,solution)
        println("t: ",t,"\t","conflits: ",solution.obj)
    end
end

t: 1	conflits: 121
t: 2	conflits: 115
t: 3	conflits: 114
t: 4	conflits: 111
t: 5	conflits: 107
t: 6	conflits: 105
t: 7	conflits: 103
t: 8	conflits: 101
t: 9	conflits: 96
t: 10	conflits: 94
t: 11	conflits: 90
t: 12	conflits: 88
t: 13	conflits: 87
t: 14	conflits: 85
t: 15	conflits: 83
t: 16	conflits: 82
t: 17	conflits: 80
t: 18	conflits: 78
t: 19	conflits: 78
t: 20	conflits: 78
t: 21	conflits: 75
t: 22	conflits: 74
t: 23	conflits: 72
t: 24	conflits: 68
t: 25	conflits: 67


# 3) Métaheuristique proposée

On choisit d'écrire un

Tabou

In [5]:
# Test de descente locale avec tabou
begin
    instance = instance_list[3]
    solution = sol_alea(instance)

    for t = 1:500
        tabu_neighbor(instance,solution,10,t)
        println("t: ",t,"\t","conflits: ",solution.obj)
    end
end

t: 1	conflits: 144
t: 2	conflits: 143
t: 3	conflits: 140
t: 4	conflits: 140
t: 5	conflits: 137
t: 6	conflits: 134
t: 7	conflits: 133
t: 8	conflits: 131
t: 9	conflits: 131
t: 10	conflits: 131
t: 11	conflits: 129
t: 12	conflits: 126
t: 13	conflits: 126
t: 14	conflits: 126
t: 15	conflits: 124
t: 16	conflits: 121
t: 17	conflits: 121
t: 18	conflits: 121
t: 19	conflits: 120
t: 20	conflits: 119
t: 21	conflits: 119
t: 22	conflits: 117
t: 23	conflits: 116
t: 24	conflits: 116
t: 25	conflits: 116
t: 26	conflits: 112
t: 27	conflits: 111
t: 28	conflits: 110
t: 29	conflits: 110
t: 30	conflits: 108
t: 31	conflits: 103
t: 32	conflits: 103
t: 33	conflits: 103
t: 34	conflits: 103
t: 35	conflits: 98
t: 36	conflits: 98
t: 37	conflits: 98
t: 38	conflits: 98
t: 39	conflits: 96
t: 40	conflits: 93
t: 41	conflits: 93
t: 42	conflits: 93
t: 43	conflits: 93
t: 44	conflits: 93
t: 45	conflits: 93
t: 46	conflits: 93
t: 47	conflits: 93
t: 48	conflits: 93
t: 49	conflits: 90
t: 50	conflits: 90
t: 51	conflits: 90
t: 52	

424	conflits: 55
t: 425	conflits: 55
t: 426	conflits: 55
t: 427	conflits: 55
t: 428	conflits: 55
t: 429	conflits: 55
t: 430	conflits: 55
t: 431	conflits: 55
t: 432	conflits: 55
t: 433	conflits: 55
t: 434	conflits: 55
t: 435	conflits: 55
t: 436	conflits: 55
t: 437	conflits: 55
t: 438	conflits: 55
t: 439	conflits: 55
t: 440	conflits: 55
t: 441	conflits: 55
t: 442	conflits: 55
t: 443	conflits: 55
t: 444	conflits: 55
t: 445	conflits: 55
t: 446	conflits: 55
t: 447	conflits: 55
t: 448	conflits: 55
t: 449	conflits: 55
t: 450	conflits: 55
t: 451	conflits: 55
t: 452	conflits: 55
t: 453	conflits: 55
t: 454	conflits: 55
t: 455	conflits: 55
t: 456	conflits: 55
t: 457	conflits: 55
t: 458	conflits: 55
t: 459	conflits: 55
t: 460	conflits: 55
t: 461	conflits: 55
t: 462	conflits: 55
t: 463	conflits: 55
t: 464	conflits: 55
t: 465	conflits: 55
t: 466	conflits: 55
t: 467	conflits: 55
t: 468	conflits: 55
t: 469	conflits: 55
t: 470	conflits: 55
t: 471	conflits: 55
t: 472	conflits: 55
t: 473	conflits: 55
t: 

Algo génétique

In [6]:
begin
    # pass
end